In [1]:
%%capture
!pip install feature-engine
!pip install optuna

In [2]:
%%capture
!pip install fastai wwf -q --upgrade

In [3]:
import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader,Dataset
from torch.autograd import Variable
import torch.optim as optim
import optuna
from optuna.integration import FastAIV2PruningCallback
from optuna.visualization import plot_optimization_history, plot_param_importances

import warnings
warnings.filterwarnings("ignore")


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sb
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import random
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
import gc

from fastai.tabular.all import *
#hide_input
from wwf.utils import state_versions
state_versions(['fastai', 'fastcore', 'wwf'])



---
This article is also a Jupyter Notebook available to be run from the top down. There
will be code snippets that you can then run in any environment.

Below are the versions of `fastai`, `fastcore`, and `wwf` currently running at the time of writing this:
* `fastai` : 2.7.11 
* `fastcore` : 1.5.28 
* `wwf` : 0.0.16 
---

In [4]:
train_raw = pd.read_csv('../wids_datathon2023/train_data.csv', parse_dates=["startdate"])
test_raw = pd.read_csv('../wids_datathon2023/test_data.csv', parse_dates=["startdate"])
submit = pd.read_csv('../wids_datathon2023/sample_solution.csv')
target = 'contest-tmp2m-14d__tmp2m'

In [5]:
def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

def location_nom(train, test):
    scale = 14

    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)

    all_df = pd.concat([train, test], axis=0)
    all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
    train = all_df.iloc[:len(train)]
    test = all_df.iloc[len(train):].drop(target, axis=1)
    
    return train, test

def categorical_encode(train, test):
    le = LabelEncoder()
    train['climateregions__climateregion'] = le.fit_transform(train['climateregions__climateregion'])
    test['climateregions__climateregion'] = le.transform(test['climateregions__climateregion'])
    return train, test

def one_hot_categorical_encode(train, test):
    ohe_enc = fe_OneHotEncoder(
    top_categories=None,
    variables=['climateregions__climateregion'],  # we can select which variables to encode
    drop_last=True)  # to return k-1, false to return k
    
    ohe_enc.fit(train)
    train = ohe_enc.transform(train)
    test = ohe_enc.transform(test)
    return train, test
    
def creat_new_featute(df):
    df['year'] = df['startdate'].dt.year
    df['month'] = df['startdate'].dt.month
    df['day_of_year'] = df['startdate'].dt.dayofyear
    # df['day_of_week'] = df['startdate'].dt.dayofweek
    # df['week_of_year'] = df['startdate'].dt.isocalendar().week
    return df

def filter_na_cols(df):
    count_na_df = df.isna().sum() 
    if count_na_df[count_na_df > 0].tolist():
        return count_na_df[count_na_df > 0]
    else:
        return 'Clean dataset'
'''
def feature_engineering(train_raw, test_raw):
    train, test = location_nom(train_raw, test_raw)
    #train = fill_na(train)
    train = creat_new_featute(train)
    test = creat_new_featute(test)
    #train, test = categorical_encode(train, test)
    drop_cols = ['index', 'startdate', 'lat', 'lon', target]
    features = [col for col in train.columns if col not in drop_cols]
    X = train[features]
    X_test = test[features]
    y = train[target]
    
    X, X_test = one_hot_categorical_encode(X, X_test)

    return X, y, X_test
'''
def feature_engineering_tabnet(train_raw, test_raw):
    train, test = location_nom(train_raw, test_raw)
    #train = fill_na(train)
    train = creat_new_featute(train)
    test = creat_new_featute(test)
    #train, test = categorical_encode(train, test)
    drop_cols = ['index', 'startdate', 'lat', 'lon', target]
    features = [col for col in train.columns if col not in drop_cols]
    X = train[features]
    X_test = test[features]
    y = train[target]
    
    #X, X_test = one_hot_categorical_encode(X, X_test)

    return X, y, X_test

In [6]:
X, y, X_test = feature_engineering_tabnet(train_raw.copy(), test_raw.copy())

In [7]:
col_na = [
    'nmme0-tmp2m-34w__ccsm30', 
    'nmme-tmp2m-56w__ccsm3', 
    'nmme-prate-34w__ccsm3', 
    'nmme0-prate-56w__ccsm30', 
    'nmme0-prate-34w__ccsm30', 
    'nmme-prate-56w__ccsm3', 
    'nmme-tmp2m-34w__ccsm3']

g_means =  ['nmme0-tmp2m-34w__nmme0mean', 
 'nmme-tmp2m-56w__nmmemean', 
 'nmme-prate-34w__nmmemean', 
 'nmme0-prate-56w__nmme0mean', 
 'nmme0-prate-34w__nmme0mean', 
 'nmme-prate-56w__nmmemean', 
 'nmme-tmp2m-34w__nmmemean']


g_1 = ['nmme0-tmp2m-34w__cancm30',
'nmme0-tmp2m-34w__cancm40',
'nmme0-tmp2m-34w__ccsm40',
'nmme0-tmp2m-34w__cfsv20',
'nmme0-tmp2m-34w__gfdlflora0',
'nmme0-tmp2m-34w__gfdlflorb0',
'nmme0-tmp2m-34w__gfdl0',
'nmme0-tmp2m-34w__nasa0']

g_2 = ['nmme-tmp2m-56w__cancm3',
'nmme-tmp2m-56w__cancm4',
'nmme-tmp2m-56w__ccsm4',
'nmme-tmp2m-56w__cfsv2',
'nmme-tmp2m-56w__gfdl',
'nmme-tmp2m-56w__gfdlflora',
'nmme-tmp2m-56w__gfdlflorb',
'nmme-tmp2m-56w__nasa']

g_3 = ['nmme-prate-34w__cancm3',
'nmme-prate-34w__cancm4',
'nmme-prate-34w__ccsm4',
'nmme-prate-34w__cfsv2',
'nmme-prate-34w__gfdl',
'nmme-prate-34w__gfdlflora',
'nmme-prate-34w__gfdlflorb',
'nmme-prate-34w__nasa']

g_4 = [ 'nmme0-prate-56w__cancm30',
'nmme0-prate-56w__cancm40',
'nmme0-prate-56w__ccsm40',
'nmme0-prate-56w__cfsv20',
'nmme0-prate-56w__gfdlflora0',
'nmme0-prate-56w__gfdlflorb0',
'nmme0-prate-56w__gfdl0',
'nmme0-prate-56w__nasa0']

g_5 = ['nmme0-prate-34w__cancm30',
'nmme0-prate-34w__cancm40',
'nmme0-prate-34w__ccsm40',
'nmme0-prate-34w__cfsv20',
'nmme0-prate-34w__gfdlflora0',
'nmme0-prate-34w__gfdlflorb0',
'nmme0-prate-34w__gfdl0',
'nmme0-prate-34w__nasa0']

g_6 = ['nmme-prate-56w__cancm3',
'nmme-prate-56w__cancm4',
'nmme-prate-56w__ccsm4',
'nmme-prate-56w__cfsv2',
'nmme-prate-56w__gfdl',
'nmme-prate-56w__gfdlflora',
'nmme-prate-56w__gfdlflorb',
'nmme-prate-56w__nasa']

g_7 = ['nmme-tmp2m-34w__cancm3',
'nmme-tmp2m-34w__cancm4',
'nmme-tmp2m-34w__ccsm4',
'nmme-tmp2m-34w__cfsv2',
'nmme-tmp2m-34w__gfdl',
'nmme-tmp2m-34w__gfdlflora',
'nmme-tmp2m-34w__gfdlflorb',
'nmme-tmp2m-34w__nasa']

In [8]:
gs = [g_1, g_2, g_3, g_4, g_5, g_6, g_7]

zip_cols = zip(col_na, gs, g_means)
for c, g, m in (zip_cols):
    X[c] = (X[m]*9) - X[g].sum(1)
X['ccsm30'] = 9*X['nmme0mean'] - (X['cancm30']+X['cancm40']+X['ccsm40']+X['cfsv20']+X['gfdlflora0']+X['gfdlflorb0']+X['gfdl0']+X['nasa0'])

filter_na_cols(X)

'Clean dataset'

In [9]:
# Add new features
X['YearsSinceRecordings'] = datetime.now().year - X.year
X_test['YearsSinceRecordings'] = datetime.now().year - X_test.year

In [10]:


def identify_correlated(df, threshold):
    corr_matrix = df.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    reduced_corr_matrix = corr_matrix.mask(mask)
    features_to_drop = [c for c in reduced_corr_matrix.columns if any(reduced_corr_matrix[c] > threshold)]
    return features_to_drop


In [11]:
features_to_drop = identify_correlated(X, .80)
X_reduced = pd.DataFrame(X.drop(features_to_drop, axis=1))
X_test_reduced = pd.DataFrame(X_test.drop(features_to_drop, axis=1))

In [12]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #os.environ['CUDA_LAUNCH_BLOCKING'] = 1
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(seed=777)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_reduced, y, test_size=0.33, random_state=42)

train_index = X_train.index
val_index = X_val.index

splits = (L(list(train_index)), L(list(val_index)))

In [14]:
train_df_fast = pd.concat([X_reduced,y],axis=1)
test_df_fast = X_test_reduced

In [15]:
categorical_features_indices = np.where((X_reduced.dtypes != np.float64))

list_cols= list(X_reduced.columns)
categ_feats = []
for i in categorical_features_indices[0]:
    categ_feats.append(list_cols[i])

cont_feats = list(X_reduced.columns).copy()
for col in categ_feats:
    cont_feats.remove(col)   

In [16]:
train_df_fast[categ_feats] = train_df_fast[categ_feats].astype('category')
test_df_fast[categ_feats] = test_df_fast[categ_feats].astype('category')

In [17]:
def rmse_fast(preds, targs):
    x = (targs-preds)
    return (x**2).mean().sqrt()

## Optuna optimization


In [37]:
dpath = Path('../fast_ai_regression/OptunaFastAi')
dpath.mkdir(exist_ok=True, parents=True)

In [38]:
learner = None

lpath = Path(dpath/"best_learner.pkl")

if lpath.exists():
    best_learner = joblib.load(lpath)
else:
    best_learner = None

In [39]:
def objective(trial: optuna.Trial):
    num_layers = trial.suggest_int('n_layers', 2, 7)
    num_layers += 1
    layers, ps = [], []
    
    # shrinking sizes for deeper layers
    pot_layers = [
        [500, 1000, 1500, 2000, 2500,  3000, 3500, 4000, 5000],
        [100, 250, 500, 750, 1000, 1500, 2000, 2500, 3000],
        [50, 100, 200, 300, 400, 500, 750, 1000, 1500, 2000],
        [50, 100, 150, 200, 300, 400, 500, 750, 1000],
        [50, 100, 200, 300, 400, 500],
        [50, 100, 200, 300, 400],
        [50, 100, 200, 300]
    ]

    # size of last layer is choosen automatically by fast.ai
    for i in range(num_layers - 1):
        num_units = trial.suggest_categorical(f'num_units_{i}', pot_layers[i])
        
        # although my inital intuition would be to reduce the dropout for deeper
        # layers, the optimization showd that this does not lead to the best results
        p = trial.suggest_uniform(f'ps_{i}', 0, 0.5)
        
        layers.append(num_units)
        ps.append(p)
    

     # to validate the results we use randomly 20% of the training set
    splits = (L(list(train_index)), L(list(val_index)))

    dls = TabularPandas(
        train_df_fast,
        cont_names=cont_feats,
        cat_names=categ_feats,
        procs=[Categorify, Normalize],
        y_names=target,
        splits=splits
    ).dataloaders(bs=1024)


    callbacks = [
        SaveModelCallback(min_delta=0.0005, monitor='_rmse', comp=np.less, fname='model_triv_best'),
        FastAIV2PruningCallback(trial, monitor='_rmse')
    ]

    # I leave the automically size for the embedding layers in place, but trial on the embbeding droppout
    emb_drop = trial.suggest_uniform('emb_drop', 0, 0.35)
    
    cfg = tabular_config(embed_p=emb_drop, ps=ps)
    global learner
    learner = tabular_learner(dls, layers=layers, metrics=[rmse], config=cfg)
    
    # could be improved by an automatic learning rate finder, fast.ai brings the capabilities for that,
    # when doing so, the learning rate would fit to the size of the network
    
    
    lr_max = trial.suggest_uniform('lr_max', 0.01, 0.2)
    weight_decay = trial.suggest_uniform('weight_decay', 0.02, 0.25)
    learner.fit_one_cycle(60, lr_max=lr_max, wd=weight_decay, cbs=callbacks)

    return learner.validate()[-1]

- This callback stores the best model in the global variable best_learner, so I can use it later without retraining.

In [40]:
def saveBestModelCallback(study, trial):
    global best_learner
    if study.best_trial == trial:
        best_learner = learner

In [41]:
spath = dpath / "study.pkl"
if spath.exists():
    study = joblib.load(spath)
else:
    study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=4, interval_steps=4))

[I 2023-03-01 05:24:26,132] A new study created in memory with name: no-name-84339e25-5d97-4735-b73e-ef5387c43a67


In [42]:
study.optimize(objective, timeout=60*60*2, callbacks=[saveBestModelCallback])

epoch,train_loss,valid_loss,_rmse,time
0,10.976829,2.128156,1.458820,00:09
1,2.785506,1.287616,1.134732,00:08
2,1.856664,1.351644,1.162602,00:08
3,1.387176,1.210426,1.100194,00:08
4,1.226804,0.688456,0.829732,00:08
5,1.406093,0.736102,0.857964,00:08
6,1.079582,0.711334,0.843406,00:08
7,1.153550,3.198082,1.788318,00:08
8,1.045149,1.037761,1.018705,00:08
9,1.291115,1.550173,1.245059,00:08


Better model found at epoch 0 with _rmse value: 1.4588199853897095.
Better model found at epoch 1 with _rmse value: 1.1347315311431885.
Better model found at epoch 3 with _rmse value: 1.1001935005187988.
Better model found at epoch 4 with _rmse value: 0.8297322988510132.
Better model found at epoch 10 with _rmse value: 0.5240662097930908.
Better model found at epoch 34 with _rmse value: 0.42367520928382874.
Better model found at epoch 47 with _rmse value: 0.31007373332977295.
Better model found at epoch 54 with _rmse value: 0.27286964654922485.
Better model found at epoch 55 with _rmse value: 0.22856423258781433.
Better model found at epoch 56 with _rmse value: 0.22346924245357513.


[I 2023-03-01 05:33:20,735] Trial 0 finished with value: 0.22346924245357513 and parameters: {'n_layers': 4, 'num_units_0': 5000, 'ps_0': 0.2665780930728035, 'num_units_1': 3000, 'ps_1': 0.4777807775360274, 'num_units_2': 750, 'ps_2': 0.22213924711751115, 'num_units_3': 750, 'ps_3': 0.2680383928571601, 'emb_drop': 0.15453664878390672, 'lr_max': 0.022830139822053893, 'weight_decay': 0.07032137320145015}. Best is trial 0 with value: 0.22346924245357513.


epoch,train_loss,valid_loss,_rmse,time
0,3.504613,0.841896,0.917549,00:04
1,1.918561,1.803373,1.342897,00:04
2,2.097572,5.189146,2.277970,00:04
3,1.494464,0.896678,0.946931,00:04
4,1.571245,2.923380,1.709789,00:04
5,1.943222,4.073289,2.018239,00:04
6,10.547284,14.670088,3.830155,00:04
7,3.348648,634.055603,25.180468,00:04
8,25.226780,5.555125,2.356931,00:04
9,2.736314,12.503782,3.536069,00:04


Better model found at epoch 0 with _rmse value: 0.9175488352775574.
Better model found at epoch 33 with _rmse value: 0.8406783938407898.
Better model found at epoch 37 with _rmse value: 0.8221500515937805.
Better model found at epoch 38 with _rmse value: 0.7407842874526978.
Better model found at epoch 50 with _rmse value: 0.5850184559822083.
Better model found at epoch 55 with _rmse value: 0.47167569398880005.
Better model found at epoch 56 with _rmse value: 0.4617711901664734.


...

Better model found at epoch 0 with _rmse value: 1.2358076572418213.
Better model found at epoch 1 with _rmse value: 1.0054264068603516.
Better model found at epoch 2 with _rmse value: 0.911983072757721.


[I 2023-03-01 07:24:26,555] Trial 25 pruned. Exception occured in `FastAIV2PruningCallback` when calling event `after_fit`:
	Trial was pruned at epoch 4.


epoch,train_loss,valid_loss,_rmse,time
0,43.848270,12.460073,3.529883,00:04
1,2.009573,21.991344,4.689493,00:04
2,1.341742,17.516718,4.185298,00:04
3,1.103649,13.503144,3.674662,00:04
4,0.974553,13.624718,3.691167,00:04


Better model found at epoch 0 with _rmse value: 3.52988338470459.


[I 2023-03-01 07:25:01,973] Trial 26 pruned. Exception occured in `FastAIV2PruningCallback` when calling event `after_fit`:
	Trial was pruned at epoch 4.


In [43]:
study.best_params

{'n_layers': 3,
 'num_units_0': 2500,
 'ps_0': 0.005264426213980364,
 'num_units_1': 1500,
 'ps_1': 0.395893391064062,
 'num_units_2': 2000,
 'ps_2': 0.2920024847325339,
 'emb_drop': 0.11260881911393272,
 'lr_max': 0.010558094918660305,
 'weight_decay': 0.12747437575952583}

In [44]:
test_dl = best_learner.dls.test_dl(test_df_fast)

preds_fast_tuned_raw = best_learner.get_preds(dl=test_dl)[0]
preds_fast_tuned = preds_fast_tuned_raw.squeeze().numpy()

In [45]:
preds_fast_tuned 

array([29.067434, 29.069141, 29.136911, ...,  8.0159  ,  8.683668,
        9.296513], dtype=float32)

In [46]:
best_learner.export('../fast_ai_regression/fast_tuned_2_autresfeats')

In [48]:
test_df_fast.shape

(31354, 146)

In [19]:
# Reoload the model
learn = load_learner('../fast_ai_regression/fast_tuned_2_autresfeats')
dl_test = learn.dls.test_dl(test_df_fast)

preds_fast_raw = learn.get_preds(dl=dl_test)[0]
preds_fast = preds_fast_raw.squeeze().numpy()
preds_fast

array([29.067434, 29.06914 , 29.13691 , ...,  8.015901,  8.683669,
        9.296514], dtype=float32)

In [20]:
best622= pd.read_csv('../Pytorch&Keras/sub_pytorch/(0.622)best627_0.95_pytorch_mod3_0.05.csv')[target]

In [21]:
0.9*best622+0.1*preds_fast

0        29.248088
1        29.256530
2        29.281795
3        29.339445
4        29.416106
           ...    
31349     6.817774
31350     7.023606
31351     6.421200
31352     7.212822
31353     7.732646
Name: contest-tmp2m-14d__tmp2m, Length: 31354, dtype: float64